# Metadata JSON generator

This notebook builds pipeline metadata JSON based on SQL Server metadata stored in Parquet and configuration DataFrames.

## Parameter setup
Define the Fabric parameter used to filter metadata for a specific source.

In [ ]:
# Parameters
# This cell is tagged so Fabric pipelines can override the default value.
source = "anva_meeus"


## Imports and helper utilities
Load dependencies and helper functions for identifier sanitization.

In [ ]:
import json
import re
from pathlib import Path
from typing import Dict, List

from pyspark.sql import DataFrame, functions as F, types as T

try:
    from notebookutils import mssparkutils  # Fabric
except ImportError:  # Local Spark
    mssparkutils = None


def make_safe_identifier(name: str) -> str:
    "Return a Spark/Delta safe identifier."
    cleaned = re.sub(r"[^0-9A-Za-z_]+", "_", name.strip())
    cleaned = re.sub(r"^[^A-Za-z_]+", "", cleaned)
    cleaned = cleaned or "col"
    return cleaned


make_safe_identifier_udf = F.udf(make_safe_identifier, T.StringType())


## Define configuration DataFrames
Create in-notebook DataFrames for source mapping, disabled tables, size classes, load modes, window settings, and exclusions.

In [ ]:
# 1) Bron/server/database mapping
sources_schema = T.StructType([
    T.StructField("Bron", T.StringType(), False),
    T.StructField("Server", T.StringType(), False),
    T.StructField("Database", T.StringType(), False),
])

df_sources = spark.createDataFrame(
    [
        ("ccs_level", "vmdwhidpweu01", "InsuranceData_CCS_DWH"),
        ("anva_meeus", "vmdwhidpweu01\MEEUS", "InsuranceData_ANVA_DWH"),
        ("vizier", "viz-sql01-mi-p.1d57ac4f4d63.database.windows.net", "CRM_DWH"),
        ("ods_reports", "vmdwhodsanvpweu", "OG_ODS_Reports"),
        ("anva_concern", "vmdwhidpweu01", "InsuranceData_ANVA_DWH"),
        ("insurance_data_im", "vmdwhidpweu01", "InsuranceData_OpGroen_DWH"),
    ],
    schema=sources_schema,
)

# 2) Disabled tables
_disabled_schema = T.StructType([
    T.StructField("Bron", T.StringType(), False),
    T.StructField("schema_name", T.StringType(), False),
    T.StructField("obj_name", T.StringType(), False),
])

df_disabled_tables = spark.createDataFrame(
    [
        ("anva_concern", "dbo", "Jobmonitor"),
        ("anva_concern", "dbo", "LaatsteVerversing"),
        ("anva_concern", "dbo", "Metadata"),
        ("anva_concern", "dbo", "VrijeLabels"),
        ("anva_concern", "pbi", "Nulmeting_Clausules"),
        ("anva_concern", "pbi", "Nulmeting_CodesDekking"),
        ("anva_concern", "pbi", "Nulmeting_CodesNAW"),
        ("anva_concern", "pbi", "Nulmeting_CodesPolis"),
        ("anva_concern", "pbi", "Nulmeting_LabelDekking"),
        ("anva_concern", "pbi", "Nulmeting_LabelNAW"),
        ("anva_concern", "pbi", "Nulmeting_LabelPolis"),
        ("anva_concern", "pbi", "Nulmeting_NAWDetails"),
        ("anva_concern", "pbi", "Nulmeting_NAWLabels"),
        ("anva_concern", "pbi", "Nulmeting_PolisDetails"),
        ("anva_concern", "pbi", "Nulmeting_PolisLabels"),
        ("anva_concern", "pbi", "Nulmeting_Voorwaarden"),
        ("anva_meeus", "dbo", "Jobmonitor"),
        ("anva_meeus", "dbo", "LaatsteVerversing"),
        ("anva_meeus", "dbo", "Metadata"),
        ("anva_meeus", "dbo", "VrijeLabels"),
        ("anva_meeus", "pbi", "Nulmeting_Clausules"),
        ("anva_meeus", "pbi", "Nulmeting_CodesDekking"),
        ("anva_meeus", "pbi", "Nulmeting_CodesNAW"),
        ("anva_meeus", "pbi", "Nulmeting_CodesPolis"),
        ("anva_meeus", "pbi", "Nulmeting_LabelDekking"),
        ("anva_meeus", "pbi", "Nulmeting_LabelNAW"),
        ("anva_meeus", "pbi", "Nulmeting_LabelPolis"),
        ("anva_meeus", "pbi", "Nulmeting_NAWDetails"),
        ("anva_meeus", "pbi", "Nulmeting_NAWLabels"),
        ("anva_meeus", "pbi", "Nulmeting_PolisDetails"),
        ("anva_meeus", "pbi", "Nulmeting_PolisLabels"),
        ("anva_meeus", "pbi", "Nulmeting_Voorwaarden"),
    ],
    schema=_disabled_schema,
)

# 3) Size class
size_schema = T.StructType([
    T.StructField("Bron", T.StringType(), False),
    T.StructField("schema_name", T.StringType(), False),
    T.StructField("obj_name", T.StringType(), False),
    T.StructField("size_class", T.StringType(), False),
])

df_size_class = spark.createDataFrame(
    [
        ("anva_concern", "pbi", "Fact_PremieFacturen", "L"),
        ("ccs_level", "pbi", "Fact_PremieBoekingen", "L"),
        ("geintegreerd_model", "pbi", "Fact_PremieFacturen", "L"),
        ("anva_meeus", "pbi", "Fact_PremieFacturen", "L"),
    ],
    schema=size_schema,
)

# 4) Load mode
load_schema = T.StructType([
    T.StructField("Bron", T.StringType(), False),
    T.StructField("schema_name", T.StringType(), False),
    T.StructField("obj_name", T.StringType(), False),
    T.StructField("load_mode", T.StringType(), False),
    T.StructField("filter_column", T.StringType(), True),
    T.StructField("kind", T.StringType(), True),
])

df_load_mode = spark.createDataFrame(
    [
        ("anva_concern", "pbi", "Fact_PremieFacturen", "window", "Boek_Datum", "datetime"),
        ("ccs_level", "pbi", "Fact_PremieBoekingen", "window", "Boek_Datum", "datetime"),
        ("geintegreerd_model", "pbi", "Fact_PremieFacturen", "window", "Boek_Datum", "datetime"),
        ("anva_meeus", "pbi", "Fact_PremieFacturen", "window", "Boek_Datum", "datetime"),
        ("vizier", "dbo", "Relaties", "incremental", "Updatedatum", "stamp17"),
        ("vizier", "dbo", "Contactpersonen", "incremental", "Upd_dt", "stamp17"),
        ("vizier", "dbo", "Sleutels", "incremental", "upd", "stamp17"),
        ("vizier", "dbo", "Polissen", "incremental", "upd_dt", "stamp17"),
        ("vizier", "dbo", "Schades", "incremental", "upd_dt", "stamp17"),
        ("vizier", "dbo", "DnB", "incremental", "upd_dt", "stamp17"),
        ("vizier", "dbo", "Contactmomenten", "incremental", "Upd", "stamp17"),
        ("vizier", "dbo", "Taken", "incremental", "upd", "stamp17"),
        ("vizier", "dbo", "Sales", "incremental", "UPD_DT", "stamp17"),
        ("vizier", "dbo", "Retenties", "incremental", "UPD_DT", "stamp17"),
        ("vizier", "dbo", "Adresbeeld", "incremental", "UPD_DT", "stamp17"),
        ("vizier", "dbo", "UBO_Onderzoeken", "incremental", "UPD_DT", "stamp17"),
        ("vizier", "dbo", "Producten", "incremental", "upd", "stamp17"),
        ("vizier", "dbo", "Medewerkers", "incremental", "id_upd", "stamp17"),
        ("vizier", "dbo", "Klachten", "incremental", "upd_dt", "stamp17"),
        ("vizier", "dbo", "Verkoopkansen", "incremental", "upd", "stamp17"),
        ("vizier", "dbo", "Interesses", "incremental", "UPD_DT", "stamp17"),
    ],
    schema=load_schema,
)

# 5) Window config
window_schema = T.StructType([
    T.StructField("Bron", T.StringType(), False),
    T.StructField("schema_name", T.StringType(), False),
    T.StructField("obj_name", T.StringType(), False),
    T.StructField("partition_column", T.StringType(), False),
    T.StructField("granularity", T.StringType(), False),
    T.StructField("lookback_months", T.IntegerType(), False),
])

df_window_config = spark.createDataFrame(
    [
        ("anva_concern", "pbi", "Fact_PremieFacturen", "Boek_Datum", "month", 12),
        ("geintegreerd_model", "pbi", "Fact_PremieFacturen", "Boek_Datum", "month", 12),
        ("anva_meeus", "pbi", "Fact_PremieFacturen", "Boek_Datum", "month", 12),
        ("ccs_level", "pbi", "Fact_PremieBoekingen", "Boek_Datum", "month", 12),
    ],
    schema=window_schema,
)

# 6) Excluded tables
excluded_schema = T.StructType([
    T.StructField("Bron", T.StringType(), False),
    T.StructField("schema_name", T.StringType(), False),
    T.StructField("obj_name", T.StringType(), False),
    T.StructField("excluded", T.IntegerType(), False),
])

df_excluded_tables = spark.createDataFrame(
    [
        ("vizier", "dbo", "BO_sleutels_Wim_Verheijen", 1),
        ("vizier", "dbo", "UMG_Historie", 1),
    ],
    schema=excluded_schema,
)


## Load SQL metadata from Parquet
Resolve the source mapping and read the sqlmetadata Parquet file for the chosen source.

In [ ]:
# Resolve source mapping and load dbo.sqlmetadata from Parquet
source_row = df_sources.filter(F.col("Bron") == source).limit(1).collect()
if not source_row:
    raise ValueError(f"Unknown source '{source}' - check df_sources")

server = source_row[0]["Server"]
database = source_row[0]["Database"]

metadata_path = "Files/metadata/sqlmetadata.parquet"
sql_metadata = (
    spark.read.format("parquet")
    .load(metadata_path)
    .filter(F.col("server_name") == server)
    .filter(F.col("db_name") == database)
)


## Build base queries with type mapping
Construct SELECT projections with explicit type mappings and sanitized column aliases.

In [ ]:
# Build base_query strings with explicit type mappings

def build_projection(row: T.Row) -> str:
    col = row["column_name"]
    dtype = (row["data_type"] or "").lower()
    precision = row["numeric_precision"]
    scale = row["numeric_scale"]
    safe = make_safe_identifier(col)

    if dtype in {"decimal", "numeric"} and precision is not None and scale is not None:
        expr = f"CAST([{col}] AS decimal({precision},{scale}))"
    elif dtype == "money":
        expr = f"CAST([{col}] AS decimal(19,4))"
    elif dtype == "smallmoney":
        expr = f"CAST([{col}] AS decimal(10,4))"
    elif dtype == "tinyint":
        expr = f"CAST([{col}] AS smallint)"
    elif dtype in {"smallint", "int", "bigint", "bit", "float", "real"}:
        expr = f"CAST([{col}] AS {dtype})"
    elif dtype == "date":
        expr = f"CAST([{col}] AS date)"
    elif dtype == "datetime":
        expr = f"CAST([{col}] AS datetime2(3))"
    elif dtype == "smalldatetime":
        expr = f"CAST([{col}] AS datetime2(0))"
    elif dtype == "datetime2":
        expr = f"CAST([{col}] AS datetime2(6))"
    elif dtype == "time":
        expr = f"CONVERT(varchar(8), [{col}], 108)"
    elif dtype == "datetimeoffset":
        expr = f"CAST(SWITCHOFFSET([{col}], '+00:00') AS datetime2(6))"
    elif dtype in {"char", "varchar", "nchar", "nvarchar"}:
        expr = f"[{col}]"
    elif dtype == "text":
        expr = f"CONVERT(varchar(max), [{col}])"
    elif dtype == "ntext":
        expr = f"CONVERT(nvarchar(max), [{col}])"
    elif dtype in {"binary", "varbinary"}:
        expr = f"[{col}]"
    elif dtype == "image":
        expr = f"CONVERT(varbinary(max), [{col}])"
    elif dtype in {"rowversion", "timestamp"}:
        expr = f"CAST([{col}] AS varbinary(8))"
    elif dtype == "uniqueidentifier":
        expr = f"CONVERT(varchar(36), [{col}])"
    elif dtype == "xml":
        expr = f"CONVERT(nvarchar(max), [{col}])"
    elif dtype == "hierarchyid":
        expr = f"{col}.ToString()"
    elif dtype in {"geometry", "geography"}:
        expr = f"{col}.STAsBinary()"
    elif dtype == "sql_variant":
        expr = f"CONVERT(nvarchar(max), [{col}])"
    else:
        expr = f"[{col}]"

    return f"{expr} AS [{safe}]"


metadata_filtered = sql_metadata.select(
    "schema_name",
    "obj_name",
    "column_name",
    "data_type",
    "numeric_precision",
    "numeric_scale",
    "ordinal_position",
).orderBy("schema_name", "obj_name", "ordinal_position")

base_query_rows: Dict[str, Dict[str, List[str]]] = {}
for row in metadata_filtered.collect():
    schema = row["schema_name"]
    table = row["obj_name"]
    projection = build_projection(row)
    base_query_rows.setdefault(schema, {}).setdefault(table, []).append(projection)

base_query_records = []
for schema, tables in base_query_rows.items():
    for table, columns in tables.items():
        select_list = ", ".join(columns)
        base_query = f"SELECT {select_list} FROM [{schema}].[{table}]"
        base_query_records.append((schema, table, source, base_query))

base_query_df = spark.createDataFrame(
    base_query_records,
    schema=T.StructType(
        [
            T.StructField("schema_name", T.StringType(), False),
            T.StructField("obj_name", T.StringType(), False),
            T.StructField("Bron", T.StringType(), False),
            T.StructField("base_query", T.StringType(), False),
        ]
    ),
)


## Merge configuration for all tables
Combine base queries with table-level configuration to prepare the metadata records.

In [ ]:
# Merge configuration

tables = (
    base_query_df.alias("bq")
    .join(df_disabled_tables.alias("dis"), ["Bron", "schema_name", "obj_name"], "left")
    .join(df_size_class.alias("sz"), ["Bron", "schema_name", "obj_name"], "left")
    .join(df_load_mode.alias("lm"), ["Bron", "schema_name", "obj_name"], "left")
    .join(df_window_config.alias("wnd"), ["Bron", "schema_name", "obj_name"], "left")
    .join(df_excluded_tables.alias("ex"), ["Bron", "schema_name", "obj_name"], "left")
    .withColumn("enabled", F.when(F.col("dis.obj_name").isNull(), F.lit(True)).otherwise(F.lit(False)))
    .withColumn("size_class", F.when(F.col("sz.size_class").isNull(), F.lit("S")).otherwise(F.col("sz.size_class")))
    .withColumn("load_mode", F.when(F.col("lm.load_mode").isNull(), F.lit("snapshot")).otherwise(F.col("lm.load_mode")))
    .withColumn("excluded", F.when(F.col("ex.excluded").isNull(), F.lit(0)).otherwise(F.col("ex.excluded")))
    .withColumnRenamed("bq.base_query", "base_query")
    .select(
        F.col("bq.obj_name").alias("name"),
        "schema_name",
        "Bron",
        "enabled",
        "size_class",
        "load_mode",
        "base_query",
        F.col("lm.filter_column"),
        F.col("lm.kind"),
        F.col("wnd.partition_column"),
        F.col("wnd.granularity"),
        F.col("wnd.lookback_months"),
        "excluded",
    )
    .filter(F.col("excluded") == 0)
    .orderBy("name")
)

tables.cache()


## Assemble and persist metadata JSON
Build the final JSON payload and write it to the Files/config directory.

In [ ]:
# Assemble JSON payload

def table_record(row: T.Row) -> Dict:
    record = {
        "name": row["name"],
        "enabled": bool(row["enabled"]),
        "size_class": row["size_class"],
        "load_mode": row["load_mode"],
        "delta_schema": row["Bron"],
        "delta_table": row["name"],
        "base_query": row["base_query"],
    }
    if row["load_mode"] == "window" and row["partition_column"]:
        record["window"] = {
            "partition_column": row["partition_column"],
            "granularity": row["granularity"],
            "lookback_months": int(row["lookback_months"]),
        }
    if row["load_mode"] == "incremental" and row["filter_column"]:
        record["incremental_column"] = {
            "name": row["filter_column"],
            "kind": row["kind"],
        }
    return record


payload = {
    "source": source,
    "run_date_utc": None,
    "watermarks_path": "config/watermarks.json",
    "base_files": "greenhouse_sources",
    "connection_name": f"connection_{source}_prod",
    "defaults": {
        "concurrency_large": 2,
        "concurrency_small": 8,
        "max_rows_per_file_large": 15000000,
        "max_rows_per_file_small": 1000000,
    },
    "tables": [table_record(row) for row in tables.collect()],
}

payload_path = Path(f"Files/config/{source}_metadata.json")
payload_path.parent.mkdir(parents=True, exist_ok=True)

json_text = json.dumps(payload, ensure_ascii=False, indent=4)

if mssparkutils is not None:
    mssparkutils.fs.put(str(payload_path), json_text, overwrite=True)
else:
    with open(payload_path, "w", encoding="utf-8") as fp:
        fp.write(json_text)

print(f"Written metadata to {payload_path}")
